In [11]:
import sys
sys.path.append('/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/')
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path
import yfinance as yf
import time
from datetime import timedelta, datetime
from systems.performance_reporter import StrategyReporter, PerformanceReporter
from systems.utils import MarketDataExtractor

In [12]:
market_data_extractor = MarketDataExtractor()
strategy_reporter = StrategyReporter()
performance_reporter = PerformanceReporter(market_data_extractor)

In [13]:
# All the Inputs for the Backtest that needs to be analyzed
testname = 'unknown_strategy_20250210_212900' # Profits not Reinvested
# testname = '3aa6bc1bc58ab86f2a23419d3737ec62991783e80d8d8802f5cc68a1cfb0a33b' # Profits Reinvested & Margin Deployed @ 0.65 of 3.3x
save_folder = "/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/db/data/yahoo/1d"
test_folder_path = f'/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/db/vault/backtest_reports/{testname}'
backtest_output = strategy_reporter.load_backtest_output(test_folder_path)
open_signals = backtest_output['open_signals']
closed_signals = backtest_output['closed_signals']
config_dict = backtest_output['config_dict']

In [14]:
dict(closed_signals[0])

{'strategy_name': 'crypto_momentum_last',
 'signal_id': '64af8cd10937467b92aba2acd8ceebe3',
 'timestamp': Timestamp('2023-01-05 23:15:00+0000', tz='UTC'),
 'orders': [Order(symbol='XRPUSD', orderQuantity=29655.990510083036, orderDirection='SELL', order_type='MARKET', price=None, symbol_ltp={Timestamp('2023-01-05 23:15:00+0000', tz='UTC'): 0.3372}, timeInForce='DAY', status='closed', filled_price=0.3372, filled_timestamp=Timestamp('2023-01-05 23:15:00+0000', tz='UTC'), order_id='d2ba3d95e1214e47b96ab64f28c852d7', entryOrderBool=True, broker_order_id=None, history=[Order(symbol='XRPUSD', orderQuantity=29655.990510083036, orderDirection='SELL', order_type='MARKET', price=None, symbol_ltp={Timestamp('2023-01-05 23:15:00+0000', tz='UTC'): 0.3372}, timeInForce='DAY', status='pending', filled_price=None, filled_timestamp=None, order_id='d2ba3d95e1214e47b96ab64f28c852d7', entryOrderBool=True, broker_order_id=None, history=[], message=None, brokerage_fee_abs=None, slippage_abs=None, fresh_updat

In [15]:
win_list = []
loss_list = []
win_with_fees_list = []
loss_with_fees_list = []

for signal in closed_signals:
    if signal.pnl > 0:
        win_list.append(signal.pnl)
    else:
        loss_list.append(signal.pnl)
        
    if signal.pnl_with_fee_and_slippage > 0:
        win_with_fees_list.append(signal.pnl_with_fee_and_slippage)
    else:
        loss_with_fees_list.append(signal.pnl_with_fee_and_slippage)

In [16]:
len(win_list), len(loss_list), len(win_with_fees_list), len(loss_with_fees_list)

(41, 60, 17, 84)

In [6]:
signal_num = 1000
open_signals[signal_num].orders[0].filled_price, closed_signals[signal_num].orders[1].filled_price

(86.42, 31.2519)

In [9]:
closed_signals[signal_num].dict()

/var/folders/d0/_rd9vn610y36nmcvbvp1xr480000gn/T/ipykernel_41757/3475615908.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  closed_signals[signal_num].dict()
/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/venvs/mathematricks_venv/lib/python3.10/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `dict[str, any]` but got `Order` with value `Order(symbol='ATOMUSD', o...h_fee_and_slippage=None)` - serialized value may not be as expected
  Expected `dict[str, any]` but got `Order` with value `Order(symbol='ATOMUSD', o...h_fee_and_slippage=None)` - serialized value may not be as expected
  Expected `dict[str, any]` but got `Order` with value `Order(symbol='ATOMUSD', o...h_fee_and_slippage=None)` - serialized value may not be as expected
  return self.__pydantic_ser

{'strategy_name': 'crypto_reversion',
 'signal_id': '4ad195d2c4f54b638c2550f15d456375',
 'timestamp': Timestamp('2022-03-01 06:53:00+0000', tz='UTC'),
 'orders': [{'symbol': 'ATOMUSD',
   'orderQuantity': 1,
   'orderDirection': 'BUY',
   'order_type': 'MARKET',
   'price': None,
   'symbol_ltp': {Timestamp('2022-03-01 06:53:00+0000', tz='UTC'): 31.6508},
   'timeInForce': 'DAY',
   'status': 'closed',
   'filled_price': 31.6508,
   'filled_timestamp': Timestamp('2022-03-01 06:53:00+0000', tz='UTC'),
   'order_id': '3cf0fdf3c6e94448abab9486b19b3cbe',
   'entryOrderBool': True,
   'broker_order_id': None,
   'history': [{'symbol': 'ATOMUSD',
     'orderQuantity': 1,
     'orderDirection': 'BUY',
     'order_type': 'MARKET',
     'price': None,
     'symbol_ltp': {Timestamp('2022-03-01 06:53:00+0000', tz='UTC'): 31.6508},
     'timeInForce': 'DAY',
     'status': 'pending',
     'filled_price': None,
     'filled_timestamp': None,
     'order_id': '3cf0fdf3c6e94448abab9486b19b3cbe',
    

In [16]:
len(open_signals[signal_num].orders)

6

In [23]:
for order in open_signals[signal_num].orders:
    print(f"Symbol: {order.symbol}, Status: {order.status}, Qty: {order.orderQuantity}, Price: {order.filled_price if order.filled_price else 'N/A'}, Side: {order.orderDirection}, Type: {order.order_type}")

Symbol: BAC, Status: closed, Qty: 1241, Price: 11.43, Side: BUY, Type: MARKET
Symbol: BAC, Status: closed, Qty: 1241, Price: 11.224538976005489, Side: SELL, Type: STOPLOSS
Symbol: BAC, Status: pending, Qty: 1241, Price: N/A, Side: SELL, Type: LIMIT
Symbol: JPM, Status: closed, Qty: 281, Price: 24.91, Side: SELL, Type: MARKET
Symbol: JPM, Status: closed, Qty: 281, Price: 25.360981909951008, Side: BUY, Type: STOPLOSS
Symbol: JPM, Status: pending, Qty: 281, Price: N/A, Side: BUY, Type: LIMIT


In [4]:
#Processing Open order to make them close, and remove history
open_orders_list = strategy_reporter.clean_up_order_list(open_orders, drop_history=True)
#Processing Closed orders to remove history
closed_orders_list = strategy_reporter.clean_up_order_list(closed_orders, drop_history=True)
#Merge both orders
final_list = closed_orders_list + open_orders_list
# Step 2: Sort the filtered orders by exit order `filled_timestamp`
sorted_orders = sorted(final_list, key=lambda x: x[1]['filled_timestamp'])
# Step 3: Fetch the first and last `filled_timestamp` with buffer
start_date = sorted_orders[0][1]['filled_timestamp'] - timedelta(days=365)
end_date = sorted_orders[-1][1]['filled_timestamp'] + timedelta(days=365)
# end_date = pd.Timestamp(datetime(2010, 1, 1))
# Current we are downloading csv file for below three index
indices = ["^DJI", "^IXIC", "^GSPC"]

index_data_dict = strategy_reporter.load_and_save_index_data(indices, start_date, end_date, save_folder)
rolling_window = int(max(60, (end_date - start_date).days / 20))
print({f"Analyzing with a rolling window of {rolling_window} days"})

NameError: name 'open_orders' is not defined

### This below function calculates profit/loss from the orders  **(exitPrice - entryPrice) * orderQuantity**

This function also plot the progress lines for 3 indexes provided earlier.

In [ ]:
starting_capital = 72267.75
trade_df_by_date = strategy_reporter.create_data_input_for_cumulative_returns_and_indices(final_list, index_data_dict, starting_capital)
strategy_reporter.plot_cumulative_returns_and_indices(trade_df_by_date, index_data_dict)

In [ ]:
sorted_orders[0]

# Plot Rolling charts

In [37]:
pair_trading_orders_dict = {}

for order_history in closed_orders:
    if order_history[0]['strategy_inputs']['strategy_order_id'] not in pair_trading_orders_dict.keys():
        pair_trading_orders_dict[order_history[0]['strategy_inputs']['strategy_order_id']] = []
    pair_trading_orders_dict[order_history[0]['strategy_inputs']['strategy_order_id']].append(
        {'symbol':order_history[0]['symbol'], 
         'direction':order_history[0]['orderDirection'],
         'entry_price':order_history[0]['fill_price'], 
         'exit_price':order_history[-1]['fill_price'], 
         'order_quantity':order_history[0]['orderQuantity'],
         'entry_timestamp':order_history[0]['filled_timestamp'], 
         'symbol_ltp-len':len(order_history[-1]['symbol_ltp']),
         'exit_timestamp':order_history[-1]['filled_timestamp'], 
         'order_value': (order_history[0]['fill_price']) * (1 if order_history[0]['orderDirection'] == 'BUY' else -1) * (order_history[0]['orderQuantity']),
         'profit': ((order_history[-1]['fill_price'] - order_history[0]['fill_price']) * (1 if order_history[0]['orderDirection'] == 'BUY' else -1) * (order_history[0]['orderQuantity']))
         })

In [38]:
pair_trading_orders_dict_ = {}
count = 0
for key, value in pair_trading_orders_dict.items():
    if len(value) == 2:
        # print(len(value))
        pair_trading_orders_dict_[key] = value
pair_trading_orders_dict = pair_trading_orders_dict_

In [ ]:
both_profit_count = 0
both_loss_count = 0
mixed_count = 0
profit_count = 0
loss_count = 0
profit_pct_list = []
loss_pct_list = []

for key, value in pair_trading_orders_dict.items():
    order_1, order_2 = value
    order_1_value = abs(order_1['order_value'])
    order_2_value = abs(order_2['order_value'])
    order_1_hold_dur = order_1['symbol_ltp-len']
    order_2_hold_dur = order_2['symbol_ltp-len']
    
    order_1_profit = order_1['profit']
    order_2_profit = order_2['profit']
    total_profit = order_1_profit + order_2_profit
    if order_1_profit > 0 and order_2_profit > 0:
        both_profit_count += 1
    elif order_1_profit < 0 and order_2_profit < 0:
        both_loss_count += 1
    else:
        mixed_count += 1
    
    if total_profit > 0:
        profit_count += 1
    else:
        loss_count += 1
    pnl = total_profit/(order_1_value + order_2_value) * 100
    
    if pnl > 0:
        profit_pct_list.append(pnl)
    else:
        loss_pct_list.append(pnl)
    print(f"Metrics: order_1: {round(order_1_profit, 1)}, Order 2 PnL: {round(order_2_profit, 1)}, Total PnL: {round(total_profit, 1)}, Total Order Value: {round(order_1_value + order_2_value, 2)}, PnL %: {round(pnl, 1)}%, Order Hold Duration: {(order_1_hold_dur, order_2_hold_dur)}")

In [ ]:
from numpy import average
average(profit_pct_list), average(loss_pct_list), len(profit_pct_list), len(loss_pct_list)

In [ ]:
average(profit_pct_list)/average(loss_pct_list)

In [ ]:
both_profit_count, both_loss_count, mixed_count, profit_count, loss_count

In [ ]:
profit_count / (profit_count + loss_count)

In [ ]:
pair_trading_orders_dict

In [ ]:
closed_orders[0]